# Read Data, Settings and Parameters

In [1]:
import pandas as pd
from bson import ObjectId
from pymongo import MongoClient
from datetime import datetime, timedelta, timezone
import numpy as np

In [2]:
# for Querying MongoDB
beginning_date = datetime(2024, 9, 21, 20, 30)
finishing_date = datetime.now()
module_name = "OnlineShopping"

In [3]:
# Set Parameters Before Run
start_train_date_filter = pd.to_datetime('2024-09-22')
window_table_in_months = 3

In [4]:
# pipeline = [
#     {
#         "$match": {
#             "$and": [
#                 {"created_at": {"$gte": beginning_date, '$lt': finishing_date}},
#                 {"status": "finished"},
#                 {
#                     "$or": [
#                         {"deleted_at": {"$exists": False}},
#                         {"deleted_at": {"$exists": True, "$eq": None}},
#                     ]
#                 },
#                 {"module": module_name},
#             ]
#         }
#     },
#     {
#         "$project": {
#             "_id": 1,
#             "user_id": 1,
#             "module": 1,
#             "type": 1,
#             "product_name": {
#                 "$cond": {
#                     "if": {"$eq": ["$module", "Giftcard"]},
#                     "then": {
#                         "$concat": [
#                             {"$ifNull": ["$product.name", ""]},
#                             " ",
#                             {"$toString": {"$ifNull": ["$product.price", ""]}},
#                             " ",
#                             {"$ifNull": ["$product.country_name", ""]},
#                         ]
#                     },
#                     "else": "$product_name",
#                 }
#             },
#             "total_payment_price": "$initial_total",
#             "product_price": 1,
#             "qty": 1,
#             "date": {
#                 "$dateToString": {
#                     "format": "%Y-%m-%d",
#                     "date": {"$add": ["$created_at", 3.5 * 60 * 60 * 1000]},
#                 }
#             },
#             "client_type": 1,
#             "module_unit_price": 1,
#         }
#     },
# ]
#
# result = db.orders.aggregate(pipeline)
#
# df_orders = pd.DataFrame(result)

In [5]:
# pipeline = [
#     {
#         "$match": {
#             "$and": [
#                 {"status": "finished"},
#                 {
#                     "$or": [
#                         {"deleted_at": {"$exists": False}},
#                         {"deleted_at": {"$exists": True, "$eq": None}},
#                     ]
#                 },
#                 {"module": module_name}
#             ]
#         },
#
#     },
#     {
#         "$project": {
#             "_id": 0,
#             "user_id": 1,
#             "created_at": 1
#         }
#     },
# ]
#
# result = db.orders.aggregate(pipeline)
# df_min_order_date = pd.DataFrame(result)
#
# df_min_order_date = \
#     (df_min_order_date
#      .assign(date=lambda _df: _df["created_at"] + pd.Timedelta(hours=3, minutes=30))
#      .groupby(["user_id"]).agg(
#         firstdate=("date", "min")).reset_index()
#      .assign(firstdate=lambda _df: _df['firstdate'].dt.normalize())
#      .assign(user_id=lambda _df: _df['user_id'].astype('str'))
#      )

In [6]:
df_orders = pd.read_parquet('df_orders.parquet')
df_min_order_date = pd.read_parquet('df_min_order_date.parquet')
df_dim_date = pd.read_parquet("dimdate5.parquet") # to attach shamsi date
rfm_labels = pd.read_excel("OnlineShoppingLabelsUpdate.xlsx", engine='openpyxl') # RFM Label for any modules

In [7]:
# Define cluster categories

low_valued_clusters_labelFa = (
    rfm_labels.loc[rfm_labels['LowOrHigh'] == 'low', 'clusters_labelFa']
    .dropna()
    .unique()
    .tolist()
)

medium_valued_clusters_labelFa = (
    rfm_labels.loc[rfm_labels['LowOrHigh'] == 'medium', 'clusters_labelFa']
    .dropna()
    .unique()
    .tolist()
)


high_valued_clusters_labelFa = (
    rfm_labels.loc[rfm_labels['LowOrHigh'] == 'high', 'clusters_labelFa']
    .dropna()
    .unique()
    .tolist()
)

In [8]:

# add shamsi date to df_orders
df_min_order_date = \
    (df_min_order_date
     .assign(date=lambda _df: _df["created_at"] + pd.Timedelta(hours=3, minutes=30))
     .groupby(["user_id"]).agg(
        firstdate=("date", "min")).reset_index()
     .assign(firstdate=lambda _df: _df['firstdate'].dt.normalize())
     .assign(user_id=lambda _df: _df['user_id'].astype('str'))
     )

# add ShamsiDate to dim_date_join
df_dim_date_join = (
    df_dim_date
    .assign(miladi_d=lambda df: pd.to_datetime(df['miladi_d'], errors='coerce'))
    [['miladi_d', 'jalali_1_s']]
    .assign(
        ShamsiDate=lambda df: pd.to_numeric(
            df['jalali_1_s'].astype(str).str.replace('/', ''),
            errors='coerce'
        )
    )
    .rename(columns={'miladi_d': 'date'})
    .drop(columns=['jalali_1_s'])
    [['date', 'ShamsiDate']]
    .assign(date=lambda df: pd.to_datetime(df["date"]).dt.normalize())
)

In [9]:
# add shamsi first date to df_min_date
df_min_order_date_clean = \
    (df_min_order_date
     .assign(firstdate=lambda df: pd.to_datetime(df['firstdate']).dt.normalize())
     .merge(df_dim_date_join, left_on="firstdate", right_on="date", how='left')
     .rename(columns={'ShamsiDate': 'shamsifirstdate'})
     .drop(columns=['date'])
     )
# add ShamsiMonth column to df_raw_data
ShamsiMonthRef = [int(f"{year}{month:02}") for year in range(1401, 1410) for month in range(1, 13)]
df_orders = \
    (df_orders
     .assign(date=lambda df: pd.to_datetime(df['date'], errors='coerce'))
     .rename(columns={"_id": "factor_id"})
     .assign(factor_id=lambda df: df['factor_id'].apply(lambda x: str(x) if isinstance(x, ObjectId) else x))
     .drop(columns=["module"])
     .loc[lambda df: df['date'] >= start_train_date_filter]
     .merge(df_dim_date_join, on="date", how="left")
     .assign(ShamsiMonth=lambda df: df['ShamsiDate'].astype(str).str[:6].astype(int))
     )

In [10]:
# Determine min and max ShamsiMonth for filtering
min_shamsi_month = df_orders['ShamsiMonth'].min()

# Filter df_orders to include only rows with ShamsiMonth within the specified window
if int(f"{min_shamsi_month}01") not in df_orders['ShamsiDate'].values:
    idx = ShamsiMonthRef.index(min_shamsi_month)
    min_shamsi_month = ShamsiMonthRef[idx + 1]


max_shamsi_month = df_orders['ShamsiMonth'].max()
idx = ShamsiMonthRef.index(max_shamsi_month)
max_shamsi_month = ShamsiMonthRef[idx - 1]


df_orders = (
    df_orders
    .loc[lambda df: df["ShamsiMonth"] >= min_shamsi_month]
    .loc[lambda df: df["ShamsiMonth"] <= max_shamsi_month]
    .merge(df_min_order_date_clean, on="user_id", how="left")
)

In [11]:
df_orders_maxshamsidate = df_orders.groupby("ShamsiMonth")["ShamsiDate"].max().reset_index().rename(columns={"ShamsiDate": "max_shamsi_date"})

In [12]:
# create df_orders_with_period
df_orders_with_period = (
    pd.concat(
        [
            df_orders.loc[df_orders["ShamsiMonth"].isin(
                ShamsiMonthRef[idx: idx + window_table_in_months])].assign(
                period=(0 if idx == ShamsiMonthRef.index(min_shamsi_month) else idx - ShamsiMonthRef.index(
                    min_shamsi_month))
            )
            for idx in range(ShamsiMonthRef.index(min_shamsi_month),
                             len(ShamsiMonthRef) - window_table_in_months + 1)
            if (
            (lambda wd: len(wd) > 0 and wd["ShamsiMonth"].nunique() == window_table_in_months)(
                df_orders.loc[
                    df_orders["ShamsiMonth"].isin(ShamsiMonthRef[idx: idx + window_table_in_months])]
            )
        )
        ],
        ignore_index=True
    )
)

# calculate average days between factors index
df_orders_with_period = (
    df_orders_with_period
    .sort_values(by=["user_id", "period", "date"])
    .assign(
        days_diff=lambda df: df.groupby(["user_id", "period"])["date"].diff().dt.days
    )
    .assign(
        avg_days_between_factors=lambda df: df.groupby(["user_id", "period"])["days_diff"].transform("mean")
    )
)

In [13]:
# make a copy of rfm_labels as rfm_labels_main to keep main labels
rfm_labels_main = (
    rfm_labels
    .drop_duplicates()
    .rename(columns={'clusters_labelFa': 'clusters_labelFa_main'})
)

In [14]:
# add NewUserLabelAcc to df_raw_data_with_period for new users but new users labeled
period_ranges = (
    df_orders_with_period
    .groupby("period")["ShamsiDate"]
    .agg(["min", "max"])
    .reset_index()
    .rename(columns= {"min": "min_date", "max": "max_date"})
)

In [15]:
# make new column NewUserLabelAcc
df_orders_with_period = (
    df_orders_with_period
    .merge(
        df_orders_with_period
        .groupby("period")["ShamsiDate"]
        .agg(["min", "max"])
        .reset_index()
        .rename(columns={"min": "min_date", "max": "max_date"}),
        on="period", how="left"
    )
    .assign(
        NewUserLabelAcc=lambda df: (
                (df["shamsifirstdate"] >= df["min_date"]) &
                (df["shamsifirstdate"] <= df["max_date"])
        ).astype(int)
    )
)


# RFM Function & Score Table

In [16]:
# Function to calculate RFM for each monthly period
def calc_rfm_monthly(period, df_orders):
    data_window = df_orders[df_orders["period"] == period].copy()

    if data_window.empty:
        return pd.DataFrame()

    grouped = (
        data_window.groupby("user_id", as_index=False)
        .agg(
            max_date=('date', 'max'),
            F=('factor_id', pd.Series.nunique),
            M=('total_payment_price', 'sum')
        )
    )

    end_date = data_window["date"].max()
    grouped["R"] = (end_date - grouped["max_date"]).dt.days + 1

    # grouped["R_Score_raw"] = pd.qcut(grouped["R"], q=5, labels=False, duplicates='drop') + 1

    # grouped['R_Score_raw'] = pd.cut(
    # grouped['R'],
    # bins=[0, 15, 30, 45, 60, float('inf')],
    # labels=[1, 2, 3, 4, 5],
    # right=True
    # ).astype(int)


    grouped['F_Score'] = pd.cut(
    grouped['F'],
    bins=[0, 1, 3, 10, 30, float('inf')],
    labels=[1, 2, 3, 4, 5],
    right=True
    ).astype(int)


    grouped["R_Score_raw"] = pd.qcut(grouped["R"], q=5, labels=False, duplicates='drop') + 1
    grouped["R_Score"] = 6 - grouped["R_Score_raw"]
    # grouped["F_Score"] = pd.qcut(grouped["F"], q=10, labels=False, duplicates='drop') + 1
    grouped["M_Score"] = pd.qcut(grouped["M"], q=5, labels=False, duplicates='drop') + 1

    grouped["period"] = period
    grouped["start_month"] = data_window["ShamsiMonth"].min()
    grouped["end_month"] = data_window["ShamsiMonth"].max()

    grouped = grouped.drop(columns=["R_Score_raw"])

    return grouped

# Rule 1 & 2 : Define ("New" + clusters_labelFA) and ("Passed" + clusters_labelFa)

In [17]:
# calc df_raw_data_with_period_NULAcc column
df_orders_with_period_NULAcc = (
    df_orders_with_period
    [['user_id', 'period', 'shamsifirstdate', 'NewUserLabelAcc']]
    .drop_duplicates()
)

In [18]:
low_high_to_new_label = {
    "low": "مشتریان بازگشته کم ارزش",
    "medium": "مشتریان بازگشته فعال",
    "high": "مشتریان بازگشته ارزشمند",
}

label_map_old = (
    rfm_labels[["clusters_labelFa", "LowOrHigh"]]
    .dropna()
    .drop_duplicates()
    .assign(
        new_label=lambda x: x["LowOrHigh"].map(low_high_to_new_label)
    )
    .set_index("clusters_labelFa")["new_label"]
    .to_dict()
)


In [19]:
# Map for Passed customers (Rul
# Map for new customers (Rule 3)
# These are users where NewUserLabelAcc == 1 in their first period
# I use "تازه" temporary and will change to "جدید" later
label_map_new = {
    "مشتریان جدید": "مشتریان جدید",
    "مشتریان جدید با پتانسیل رشد": "مشتریان جدید با پتانسیل رشد",
    "مشتریان جدید در معرض خطر" : "مشتریان جدید در معرض خطر",
    "مشتریان جدید یا جدید ارزشمند و در معرض خطر" : "مشتریان جدید یا جدید ارزشمند و در معرض خطر",
    "مشتریان جدید و ارزشمند در آستانه ریزش" : "مشتریان جدید و ارزشمند در آستانه ریزش",
    "مشتریان ارزشمند یا پتانسیل ارزشمند و در معرض خطر" : "مشتریان تازه و ارزشمند یا پتانسیل ارزشمند و در معرض خطر",
    "مشتریان ارزشمند" : "مشتریان تازه و ارزشمند",
    "مشتریان کم ارزش" : "مشتریان تازه و کم ارزش",
    "مشتریان ارزشمند در آستانه ریزش" : "مشتریان تازه و ارزشمند در آستانه ریزش",
    "مشتریان پتانسیل ارزشمند" : "مشتریان تازه و پتانسیل ارزشمند"
}

In [20]:
# Rule1 & 2 combined function
def adjust_labels_for_first_period(df):

    # 1. Find the first (minimum) period for each user
    first_periods = df.groupby("user_id")["period"].min().reset_index()
    first_periods.rename(columns={"period": "first_period"}, inplace=True)

    # 2. Merge this info back into the main dataframe
    df = df.merge(first_periods, on="user_id", how="left")

    # 3. Create a mask for rows that belong to the user's first period
    mask_first_period = df["period"] == df["first_period"]

    # --- 4. Apply Rule 1 (Old Customers) ---
    mask_old = df["NewUserLabelAcc"] == 0
    mask_apply_rule1 = (
        mask_old
        & mask_first_period
        & df["clusters_labelFa_main"].isin(label_map_old)
    )
    df.loc[mask_apply_rule1, "clusters_labelFa"] = (
        df.loc[mask_apply_rule1, "clusters_labelFa_main"]
        .map(label_map_old)
    )

    # --- 5. Apply Rule 2 (New Customers) ---
    mask_new = df["NewUserLabelAcc"] == 1
    mask_apply_rule2 = (
        mask_new
        & mask_first_period
        & df["clusters_labelFa_main"].isin(label_map_new)
    )
    df.loc[mask_apply_rule2, "clusters_labelFa"] = (
        df.loc[mask_apply_rule2, "clusters_labelFa_main"]
        .map(label_map_new)
    )

    # 6. Drop the temporary helper column
    df = df.drop(columns=["first_period"])

    return df


# Rule 3: For labels that changed to 'New', replace them with 'Low Value' or 'High Value' based on the previous period.

In [21]:
# Rule 3 revised : Define (clusters_labelFA + "Adjusted")
def adjust_labels_for_subsequent_periods(df):
    # 1. Sort the data by user and period to ensure correct shifting
    df = df.sort_values(by=["user_id", "period"]).reset_index(drop=True)

    # 2. Get the label from the *previous* period for each user
    df["prev_label"] = df.groupby("user_id")["clusters_labelFa_main"].shift(1)

    # 3. Identify the first period for each user
    first_periods = df.groupby("user_id")["period"].min()
    df["is_first_period"] = df["period"] == df["user_id"].map(first_periods)

    # 4. Define masks for each rule (only apply if not first period)
    mask_new = (
        (~df["is_first_period"]) &
        (df["clusters_labelFa_main"] == "مشتریان جدید") &
        (df["prev_label"].notna())
    )

    mask_new_potential = (
        (~df["is_first_period"]) &
        (df["clusters_labelFa_main"] == "مشتریان جدید با پتانسیل رشد") &
        (df["prev_label"].notna())
    )

    # 5. Apply new labels
    df.loc[mask_new, "clusters_labelFa_main"] = "مشتریان کم ارزش"
    df.loc[mask_new_potential, "clusters_labelFa_main"] = "مشتریان ارزشمند"

    # 6. Clean up temporary columns
    df = df.drop(columns=["prev_label", "is_first_period"])

    return df

In [22]:
# Calculate the final score table with adjusted labels
df_score_table = (
    pd.concat(
        [calc_rfm_monthly(p, df_orders_with_period)
         for p in sorted(df_orders_with_period["period"].unique())],
        ignore_index=True
    )
    .assign(
        # Create the combined RFM score (e.g., 555)
        RFMSeries=lambda df: (
            df["R_Score"].astype(str) +
            df["F_Score"].astype(str) +
            df["M_Score"].astype(str)
        ).astype(int)
    )
    # Merge with the predefined RFM labels
    # here i replace rfm_labels with rfm_labels_main to get main labels first for merges functions
    .pipe(lambda df: df.merge(rfm_labels_main, on="RFMSeries", how="left", validate='many_to_one'))
    # Merge with the NewUserLabelAcc info
    .merge(df_orders_with_period_NULAcc, on=["user_id", "period"], how="left", validate = 'many_to_one')
    # Apply the combined Rule 1 and Rule 2 adjustments
    .pipe(adjust_labels_for_first_period)

    .pipe(adjust_labels_for_subsequent_periods)
)

df_score_table['clusters_labelFa'] = df_score_table['clusters_labelFa'].fillna(df_score_table['clusters_labelFa_main'])

df_score_table.drop(columns=['clusters_labelFa_main'], inplace=True)

df_score_table = df_score_table.merge(
    rfm_labels_main[['RFMSeries', 'clusters_labelFa_main']],
    on='RFMSeries',
    how='left',
    validate='many_to_one')

df_score_table =df_score_table.sort_values(by = ['user_id', 'period'])

# calc Prev Labels columns first time

In [23]:
# Define rfm_clusters will use in previous labels calculation
df_rfm_clusters = df_score_table[["user_id", "period", "RFMSeries", "clusters_labelFa", "clusters_labelFa_main", "clusters_inds"]].copy()

In [24]:
df_rfm_clusters_prev = (
    df_score_table[["user_id", "period", "RFMSeries", "clusters_labelFa",
                    "clusters_labelFa_main",
                    "clusters_inds"]]
    .assign(period=lambda df: df["period"] + 1)
    .rename(columns={"clusters_labelFa": "prev_clusters_labelFa"})
    .rename(columns={"clusters_labelFa_main": "prev_clusters_labelFa_main"})
)

In [25]:
# add previous clusters label to current
df_rfm_clusters = (df_rfm_clusters
                .merge(df_rfm_clusters_prev[["user_id", "period", "prev_clusters_labelFa", "prev_clusters_labelFa_main"]],
                       on=["user_id", "period"],
                       how="left")
                )

In [26]:
# Add prev_clusters_labelFa to rfm_table too

df_rfm_table = df_score_table[
    ["user_id", "R", "F", "M", "period", "clusters_labelFa", "clusters_labelFa_main", "clusters_inds", "RFMSeries"]].copy()

df_rfm_table = df_rfm_table.merge(
    df_rfm_clusters[["user_id", "period", "prev_clusters_labelFa", "prev_clusters_labelFa_main"]],
    on=["user_id", "period"],
    how="left"
)

In [27]:
# calc mean of main clusters
df_cluster_means = (
    df_score_table
    .groupby(["period", "clusters_labelFa_main"]) # to avoid distortion of means by new customers
    .agg(R_mean=("R_Score", "mean"),
         F_mean=("F_Score", "mean"),
         M_mean=("M_Score", "mean"))
    .reset_index()
)

In [28]:
# 1) expand to long form (each user vs each possible target cluster in same period)
df_score_with_targets = (
    df_score_table
    .merge(
        df_cluster_means.rename(columns={"clusters_labelFa_main": "target_clusters_labelFa"}),
        on="period",
        how="left"
    )
)

In [29]:
df_score_with_targets = df_score_with_targets.assign(
    dist=lambda df: (
                            (df["R_Score"] - df["R_mean"]) ** 2
                            + (df["F_Score"] - df["F_mean"]) ** 2
                            + (df["M_Score"] - df["M_mean"]) ** 2
                    ) ** 0.5
)

df_score_with_targets = df_score_with_targets.sort_values(
    ["user_id", "target_clusters_labelFa", "period"]
)

In [30]:
df_score_with_targets = df_score_with_targets.assign(
    speed=lambda df: df.groupby(
        ["user_id", "target_clusters_labelFa"]
    )["dist"].diff(),
    acc=lambda df: df.groupby(
        ["user_id", "target_clusters_labelFa"]
    )["speed"].diff()
)

In [31]:
df_score_with_targets_temp = df_score_with_targets.copy()

# ---------------- STEP 1: Unique Table For (user_id, period) ----------------
df_user_period_labels = (
    df_score_with_targets_temp.sort_values(["user_id", "period"])
    .groupby(["user_id", "period"], as_index=False)
    .first()[["user_id", "period", "clusters_labelFa", "clusters_labelFa_main"]]
)

In [32]:
# Create shifted labels

df_user_period_labels = df_user_period_labels.sort_values(["user_id", "period"])

df_user_period_labels["prev_label_1"] = (
    df_user_period_labels.groupby("user_id")["clusters_labelFa"].shift(1)
)
df_user_period_labels["prev_label_2"] = (
    df_user_period_labels.groupby("user_id")["clusters_labelFa"].shift(2)
)

df_user_period_labels["prev_label_main_1"] = (
    df_user_period_labels.groupby("user_id")["clusters_labelFa_main"].shift(1)
)
df_user_period_labels["prev_label_main_2"] = (
    df_user_period_labels.groupby("user_id")["clusters_labelFa_main"].shift(2)
)

In [33]:
df_score_with_targets_temp = df_score_with_targets_temp.merge(
    df_user_period_labels[["user_id", "period", "prev_label_1", "prev_label_2", "prev_label_main_1", "prev_label_main_2"]],
    on=["user_id", "period"],
    how="left"
)

# Add dist, speed, acceleration and their ranks

In [34]:
# [32]
# ---------------- DISTANCE RANK ----------------
df_score_with_targets_temp["rank_dist"] = (
    df_score_with_targets_temp.groupby(["period", "clusters_labelFa", "target_clusters_labelFa"])["dist"]
    .rank(method="dense", ascending=True, na_option="bottom")
)

In [35]:

# ---------------- SPEED RANK ----------------
mask_speed = (df_score_with_targets_temp["prev_label_1"] == df_score_with_targets_temp["clusters_labelFa"]) & df_score_with_targets_temp["speed"].notna()
df_score_with_targets_temp["rank_speed"] = pd.NA
df_score_with_targets_temp.loc[mask_speed, "rank_speed"] = (
    df_score_with_targets_temp.loc[mask_speed]
    .groupby(["period", "clusters_labelFa", "target_clusters_labelFa"])["speed"]
    .rank(method="dense", ascending=True, na_option="bottom")
)

# ---------------- ACCELERATION RANK ----------------
mask_acc = (
        (df_score_with_targets_temp["prev_label_1"] == df_score_with_targets_temp["clusters_labelFa"]) &
        (df_score_with_targets_temp["prev_label_2"] == df_score_with_targets_temp["clusters_labelFa"]) &
        df_score_with_targets_temp["acc"].notna()
)
df_score_with_targets_temp["rank_acc"] = pd.NA
negative_acc = df_score_with_targets_temp.loc[mask_acc & (df_score_with_targets_temp["acc"] < 0)]

df_score_with_targets_temp.loc[negative_acc.index, "rank_acc"] = (
    negative_acc
    .groupby(["period", "clusters_labelFa", "target_clusters_labelFa"])["acc"]
    .rank(method="dense", ascending=True, na_option="bottom")
)

In [36]:
# ---------------- TYPE CAST ----------------
for col in ["rank_dist", "rank_speed", "rank_acc"]:
    df_score_with_targets_temp[col] = df_score_with_targets_temp[col].astype("Int64")

df_score_with_targets = df_score_with_targets_temp.drop_duplicates()

In [37]:
# add max shamsi month to df_score_with_targets
df_orders_with_period_merge = (
    df_orders_with_period[["period", "ShamsiMonth"]]
    .drop_duplicates()
    .groupby("period")
    .agg(max_shamsi_month=("ShamsiMonth", "max"))
    .drop_duplicates()
    .reset_index()
)

In [38]:
# replace max shamsi month to df_score_with_targets
df_score_with_targets = (
    df_score_with_targets
    .merge(
        df_orders_with_period_merge[["period", "max_shamsi_month"]],
        on="period",
        how="left"
    )
    .rename(columns={"max_shamsi_month": "ShamsiMonth"})
)

#  create df_complete_skeleton to keep all users × all periods × all target clusters

In [39]:
unique_target_clusters = rfm_labels_main['clusters_labelFa_main'].unique() # i changed rfm_labels to rfm_labels_main
all_periods = sorted(df_score_with_targets['period'].unique())

In [40]:
# Find First Period with all clusters
user_first_period = \
(
    df_score_with_targets
    .groupby('user_id')[['period']]
    .min()
    .reset_index()
    .rename(columns={'period': 'first_period'})
)

In [41]:
# Create complete skeleton: every user × every period (after their first) × every target cluster
df_complete_skeleton = []

In [42]:
# Build the complete skeleton
for _, row in user_first_period.iterrows():
    user_id = row['user_id']
    first_period = row['first_period']

    # Get all periods from first_period onwards
    relevant_periods = [p for p in all_periods if p >= first_period]

    # Create records for this user
    for period in relevant_periods:
        for target_cluster in unique_target_clusters:
            df_complete_skeleton.append({
                'user_id': user_id,
                'period': period,
                'target_clusters_labelFa': target_cluster,
            })

df_complete_skeleton = pd.DataFrame(df_complete_skeleton)

In [ ]:
# Merge with df_score_with_targets to get scores and ranks
df_complete_skeleton_first_phase_assignment = \
(
    df_complete_skeleton
    .merge(
        df_score_with_targets,
        on=['user_id', 'period', 'target_clusters_labelFa'],
        how='left'
    )
)

In [44]:
# First, let's create a helper dataframe for period-level values
# Fill start_month, end_month, ShamsiMonth based on period
df_period_values = (
    df_complete_skeleton_first_phase_assignment
    .groupby('period')
    .agg(
        start_month=('start_month', 'first'),
        end_month=('end_month', 'first'),
        ShamsiMonth=('ShamsiMonth', 'first')
    )
    .reset_index()
)

In [45]:
# Sort by user_id and period for proper shifting
df_complete_skeleton_filled = (
    df_complete_skeleton_first_phase_assignment
    .sort_values(['user_id', 'period'])
    .copy()
)

In [46]:
# Fill period-level columns first
df_complete_skeleton_filled = df_complete_skeleton_filled.merge(
    df_period_values,
    on='period',
    how='left',
    suffixes=('', '_fill')
)

In [47]:
# Update null values with filled values
df_complete_skeleton_filled['start_month'] = df_complete_skeleton_filled['start_month'].fillna(
    df_complete_skeleton_filled['start_month_fill']
)
df_complete_skeleton_filled['end_month'] = df_complete_skeleton_filled['end_month'].fillna(
    df_complete_skeleton_filled['end_month_fill']
)
df_complete_skeleton_filled['ShamsiMonth'] = df_complete_skeleton_filled['ShamsiMonth'].fillna(
    df_complete_skeleton_filled['ShamsiMonth_fill']
)

# Drop helper columns
df_complete_skeleton_filled = df_complete_skeleton_filled.drop(
    columns=['start_month_fill', 'end_month_fill', 'ShamsiMonth_fill']
)

df_complete_skeleton_filled = df_complete_skeleton_filled.sort_values(['user_id', 'period']).reset_index(drop=True)

In [48]:
# "Step 1: Filling NULL rows (lost customers)

# Compute last_valid_cluster (previous cluster for each user)
df_complete_skeleton_filled['last_valid_cluster'] = (
    df_complete_skeleton_filled
    .groupby('user_id')['clusters_labelFa_main']
    .apply(lambda x: x.shift(1).ffill())
    .reset_index(level=0, drop=True)
)

# Calc Lost Users based on last valid cluster
